In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
lstm = nn.LSTM(3, 3) #input word embedding in 3 dimension  ,hidden_state in 3 dim hint: (a) and (c)
inputs1 = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5 Tx = 5 = Ty
print(inputs1)

[tensor([[-0.5525,  0.6355, -0.3968]]), tensor([[-0.6571, -1.6428,  0.9803]]), tensor([[-0.0421, -0.8206,  0.3133]]), tensor([[-1.1352,  0.3773, -0.2824]]), tensor([[-2.5667, -1.4303,  0.5009]])]


In [3]:
# initialize the hidden state.
hidden1 = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
hidden2 = hidden1
x = 1
print("inputs = ",inputs1)
print("\nhidden = ",hidden1)
for i in inputs1:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out1, hidden1 = lstm(i.view(1, 1, -1), hidden1)
    print("\n out at iter :",x ," = " ,out1)
    print("\n hidden at iter :",x ," = " ,hidden1)
    x += 1

inputs2 = torch.cat(inputs1).view(len(inputs1), 1, -1)
# hidden2 = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state

print("\ninputs = ",inputs2)
print("\nhidden = ",hidden2)
out2, hidden2 = lstm(inputs2,hidden2) #momken t3ml lstm(inputs2) w hwa mn gwa hy3ml el hiddens
print("\nout = ",out2)
print("\n\n hidden = ",hidden2)

inputs =  [tensor([[-0.5525,  0.6355, -0.3968]]), tensor([[-0.6571, -1.6428,  0.9803]]), tensor([[-0.0421, -0.8206,  0.3133]]), tensor([[-1.1352,  0.3773, -0.2824]]), tensor([[-2.5667, -1.4303,  0.5009]])]

hidden =  (tensor([[[ 0.5438, -0.4057,  1.1341]]]), tensor([[[-1.1115,  0.3501, -0.7703]]]))

 out at iter : 1  =  tensor([[[-0.2682,  0.0304, -0.1526]]], grad_fn=<StackBackward>)

 hidden at iter : 1  =  (tensor([[[-0.2682,  0.0304, -0.1526]]], grad_fn=<StackBackward>), tensor([[[-1.0766,  0.0972, -0.5498]]], grad_fn=<StackBackward>))

 out at iter : 2  =  tensor([[[-0.5370,  0.0346, -0.1958]]], grad_fn=<StackBackward>)

 hidden at iter : 2  =  (tensor([[[-0.5370,  0.0346, -0.1958]]], grad_fn=<StackBackward>), tensor([[[-1.1552,  0.1214, -0.2974]]], grad_fn=<StackBackward>))

 out at iter : 3  =  tensor([[[-0.3947,  0.0391, -0.1217]]], grad_fn=<StackBackward>)

 hidden at iter : 3  =  (tensor([[[-0.3947,  0.0391, -0.1217]]], grad_fn=<StackBackward>), tensor([[[-1.0727,  0.1104, -0.

In [4]:
print(inputs2.shape) # tx=5,batch_size=1,word_embed_dim

torch.Size([5, 1, 3])


dy LSTM 3dya lw d5lt klma klma el out bta3 el klma hytl3 nfs el hidden(a) bta3 el klma
ama lw d5lt el gomla kolha el out hytl3 lel gomla kolha wel hidden hytl3 a5r hidden state bs
fa m3na kda en el out<ty> = a<ty> dymn

In [5]:
#model PART of speech y3ne by7dd kol klma dy noun wla verb wla DET
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
print(training_data)

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}
[(['The', 'dog', 'ate', 'the', 'apple'], ['DET', 'NN', 'V', 'DET', 'NN']), (['Everybody', 'read', 'that', 'book'], ['NN', 'V', 'DET', 'NN'])]


In [13]:
dataset = []
for x,y in training_data:
    print(x,y)
    dataset.append((torch.tensor([word_to_ix[word] for word in x]),torch.tensor([tag_to_ix[tag] for tag in y])))
print(dataset[0])

['The', 'dog', 'ate', 'the', 'apple'] ['DET', 'NN', 'V', 'DET', 'NN']
['Everybody', 'read', 'that', 'book'] ['NN', 'V', 'DET', 'NN']
(tensor([0, 1, 2, 3, 4]), tensor([0, 1, 2, 0, 1]))


In [7]:
class LSTMTagger(nn.Module):
    def __init__(self,embedd_dim,hidden_dim,vocab_size,tagset_size):
        super(LSTMTagger,self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size,embedd_dim)
        self.lstm = nn.LSTM(embedd_dim,hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim,tagset_size)
    def forward(self,x):
        inputs_seq = self.embedding(x)
        #if it was language modeling i would use _ instead of a_out
        #but here i don't want hidden states any more hint: it use new initialied hidden states a,c
        #each time because POS not need previous input it needs current input only
        a_out , _ = self.lstm(inputs_seq.view(len(x), 1, -1))
        output_tags = self.hidden2tag(a_out.view(len(x), -1))
        output_scores = F.log_softmax(output_tags,dim = 1)
        return output_scores

In [8]:
vocab_size = len(word_to_ix)
tagset_size = len(tag_to_ix)
model = LSTMTagger(EMBEDDING_DIM,HIDDEN_DIM,vocab_size,tagset_size)
optimizer = optim.SGD(model.parameters(),lr=0.1)
criterion = nn.NLLLoss()
def train_model(model,optimizer,criterion,dataset,iter = 10):
    for i in range(iter):
        for x,y in dataset:
            yhat = model(x)
            
            loss = criterion(yhat,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
trained_model = train_model(model,optimizer,criterion,dataset,iter = 300)

In [12]:
# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    print(inputs)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([0, 1, 2, 3, 4])
tensor([[-0.0471, -3.8399, -3.7075],
        [-4.2119, -0.0466, -3.4840],
        [-2.8518, -3.9280, -0.0806],
        [-0.0720, -3.2803, -3.4470],
        [-3.9195, -0.0266, -5.0474]])
